In [21]:
# Author: Tobias Gerken (gerkentx@jmu.edu)
# v0.01 = 202106025
# Script to download IGRA radisonde data and to parse data into CSVs for later use



# Dependencies 
import igra # https://github.com/MBlaschek/igra/blob/master/igra  # installed via: pip install igra
import pandas as pd
import glob
import os
import datetime

# Note IGRA also has capability to parse all soundings to xarray

In [85]:
# set working dir

os.chdir('D:\Research_Data\IGRA')
# set data director
data_path = ['D:\Research_Data\IGRA']

current_time = pd.to_datetime("now").strftime('%Y%m%d_%H%M%S')
tolerance = pd.to_timedelta('5 min')

force_download = False # Forces download of data from IGRA 

In [66]:
# Get stationlist and 
station_list = igra.download.stationlist('/tmp')
station_list.to_csv('station_list_' + current_time + '.txt')
station_list.head()

Download complete, reading table ...
Data read from: /tmp/igra2-station-list.txt
Data processed 2787


,wmo,lat,lon,alt,state,name,start,end,total
id,,,,,,,,,
ACM00078861,078861,17.1170,-61.7830,10.0,,COOLIDGE FIELD (UA),1947,1993,13896
AEM00041217,041217,24.4333,54.6500,16.0,,ABU DHABI INTERNATIONAL AIRPOR,1983,2021,37899
AEXUAE05467,,25.2500,55.3700,4.0,,SHARJAH,1935,1942,2477
AFM00040911,040911,36.7000,67.2000,378.0,,MAZAR-I-SHARIF,2010,2014,2179
AFM00040913,040913,36.6667,68.9167,433.0,,KUNDUZ,2010,2013,4540


In [86]:
server = 'https://www1.ncdc.noaa.gov/pub/data/igra/data/data-por/'

stations = {'Bismarck': 'USM00072764',
            'Aberdeen': 'USM00072659',  # need to rerun b/c error 
            'Minneapolis': 'USM00072649',
            'Quad City':'USM00074455',     
            'Omaha' : 'USM00072558',
            'Topeka': 'USM00072456',
            'Springfield':'USM00072440', 
            'Nashville':'USM00072327',
            'Little Rock': 'USM00072340',
            'Jackson':'USM00072235'
           }        

# set time period
start_time = '2010'
end_time = '2019'

stations

{'Aberdeen': 'USM00072659'}

In [114]:
# Create Processing Log and write to file 
processed_stations = station_list.loc[stations.values()].rename(columns={'start': 'record_start','end': 'record_end'})
processed_stations['start']=start_time
processed_stations['end']=end_time
processed_stations.to_csv('ExtractedProfiles' + current_time + '.csv')

# now loop over files and extract profiles 
for name, id in stations.items():
    f  = [id + '*.zip']
    d = os.sep.join(data_path + f)
    
    if not os.path.exists(name):
        os.mkdir(name)
        
    #print(name, id)
    if (not(glob.glob(d)) or force_download) :
        print(id)
        igra.download.station(id,data_path[0],server = server)
        log_name = os.sep.join(data_path + [name] + ['DownloadedIGRA_' + pd.to_datetime("now").strftime('%Y%m%d_%H%M%S') + '.csv'])

        processed_stations.loc[id].to_DataFrame.csv_to_file(log_name)

    log_name = os.sep.join(data_path + [name] + ['ExtractedProfiles_' + pd.to_datetime("now").strftime('%Y%m%d_%H%M%S') + '.csv'])
    processed_stations.loc[id].to_csv(log_name)
    
    # now loop over days 
    #print(glob.glob(d)[0])
    df, headers = igra.read.ascii_to_dataframe(glob.glob(d)[0],all_columns=True)
    
    # get list of all soundings 
    soundings = headers[start_time:end_time].index.strftime('%Y-%m-%d %H').to_list()
    soundings = headers[start_time:end_time].index

    # now loop over headers and get 
    for sounding in soundings:
        time_str = sounding.strftime('%Y%m%d_%H')
        print(name + time_str)
        f = [id +  '_' + time_str + 'Z.csv']
        out = os.sep.join(data_path + [name] + f)
        
        # add tolerance for key 
        
        with open(out,'w') as f:
            f.write(f"{','.join([name,id])}\n")
            headers[sounding-dt : sounding+dt].iloc[0,1:].to_csv(f, mode='a', header=False,index=True,line_terminator='\n')       
            df.loc[(df.index > sounding-dt) & (df.index < sounding+dt) & (df.ltyp1 !=3)].to_csv(f, mode='a', header=True, index=False,line_terminator='\n')


Aberdeen20100101_00
Aberdeen20100101_12
Aberdeen20100102_00
Aberdeen20100102_12
Aberdeen20100103_00
Aberdeen20100103_12
Aberdeen20100104_00
Aberdeen20100104_12
Aberdeen20100105_00
Aberdeen20100105_12
Aberdeen20100106_00
Aberdeen20100106_12
Aberdeen20100107_00
Aberdeen20100107_12
Aberdeen20100108_00
Aberdeen20100108_12
Aberdeen20100109_00
Aberdeen20100109_12
Aberdeen20100110_00
Aberdeen20100110_12
Aberdeen20100111_00
Aberdeen20100111_12
Aberdeen20100112_00
Aberdeen20100112_12
Aberdeen20100113_00
Aberdeen20100113_12
Aberdeen20100114_00
Aberdeen20100114_12
Aberdeen20100115_00
Aberdeen20100115_12
Aberdeen20100116_00
Aberdeen20100116_12
Aberdeen20100117_00
Aberdeen20100117_12
Aberdeen20100118_00
Aberdeen20100118_12
Aberdeen20100119_00
Aberdeen20100119_12
Aberdeen20100120_12
Aberdeen20100121_00
Aberdeen20100121_12
Aberdeen20100122_00
Aberdeen20100122_12
Aberdeen20100123_00
Aberdeen20100123_12
Aberdeen20100124_00
Aberdeen20100124_12
Aberdeen20100125_00
Aberdeen20100126_00
Aberdeen20100126_12


Aberdeen20100726_00
Aberdeen20100726_12
Aberdeen20100727_00
Aberdeen20100727_12
Aberdeen20100728_00
Aberdeen20100728_12
Aberdeen20100729_00
Aberdeen20100729_12
Aberdeen20100730_00
Aberdeen20100730_12
Aberdeen20100731_00
Aberdeen20100731_12
Aberdeen20100801_00
Aberdeen20100801_12
Aberdeen20100801_19
Aberdeen20100802_00
Aberdeen20100802_12
Aberdeen20100803_00
Aberdeen20100803_12
Aberdeen20100804_00
Aberdeen20100804_12
Aberdeen20100805_00
Aberdeen20100805_12
Aberdeen20100806_00
Aberdeen20100806_12
Aberdeen20100807_00
Aberdeen20100807_12
Aberdeen20100808_00
Aberdeen20100808_12
Aberdeen20100809_00
Aberdeen20100809_12
Aberdeen20100810_00
Aberdeen20100810_12
Aberdeen20100811_00
Aberdeen20100811_12
Aberdeen20100812_00
Aberdeen20100812_12
Aberdeen20100813_00
Aberdeen20100813_12
Aberdeen20100814_00
Aberdeen20100814_12
Aberdeen20100815_00
Aberdeen20100815_12
Aberdeen20100816_00
Aberdeen20100816_12
Aberdeen20100817_00
Aberdeen20100817_12
Aberdeen20100818_00
Aberdeen20100818_12
Aberdeen20100819_00


Aberdeen20110219_00
Aberdeen20110219_12
Aberdeen20110220_00
Aberdeen20110220_12
Aberdeen20110221_00
Aberdeen20110221_12
Aberdeen20110222_00
Aberdeen20110222_12
Aberdeen20110223_00
Aberdeen20110223_12
Aberdeen20110224_00
Aberdeen20110224_12
Aberdeen20110225_00
Aberdeen20110225_12
Aberdeen20110226_00
Aberdeen20110226_12
Aberdeen20110227_00
Aberdeen20110227_12
Aberdeen20110228_00
Aberdeen20110228_12
Aberdeen20110301_00
Aberdeen20110301_12
Aberdeen20110302_00
Aberdeen20110302_12
Aberdeen20110303_00
Aberdeen20110303_12
Aberdeen20110304_00
Aberdeen20110304_12
Aberdeen20110305_00
Aberdeen20110305_12
Aberdeen20110306_00
Aberdeen20110306_12
Aberdeen20110307_00
Aberdeen20110307_12
Aberdeen20110308_00
Aberdeen20110308_12
Aberdeen20110309_00
Aberdeen20110309_12
Aberdeen20110310_00
Aberdeen20110310_12
Aberdeen20110311_00
Aberdeen20110311_12
Aberdeen20110312_12
Aberdeen20110313_00
Aberdeen20110313_12
Aberdeen20110314_00
Aberdeen20110314_12
Aberdeen20110315_00
Aberdeen20110315_12
Aberdeen20110316_00


Aberdeen20110909_12
Aberdeen20110910_00
Aberdeen20110910_12
Aberdeen20110911_00
Aberdeen20110911_12
Aberdeen20110912_00
Aberdeen20110912_12
Aberdeen20110913_00
Aberdeen20110913_12
Aberdeen20110914_00
Aberdeen20110914_12
Aberdeen20110915_00
Aberdeen20110915_12
Aberdeen20110916_00
Aberdeen20110916_12
Aberdeen20110917_00
Aberdeen20110917_12
Aberdeen20110918_00
Aberdeen20110918_12
Aberdeen20110919_00
Aberdeen20110919_12
Aberdeen20110920_00
Aberdeen20110920_12
Aberdeen20110921_00
Aberdeen20110921_12
Aberdeen20110922_00
Aberdeen20110922_12
Aberdeen20110923_00
Aberdeen20110923_12
Aberdeen20110924_00
Aberdeen20110924_12
Aberdeen20110925_00
Aberdeen20110925_12
Aberdeen20110926_00
Aberdeen20110926_12
Aberdeen20110927_00
Aberdeen20110927_12
Aberdeen20110928_00
Aberdeen20110928_12
Aberdeen20110929_00
Aberdeen20110929_12
Aberdeen20110930_00
Aberdeen20110930_12
Aberdeen20111001_00
Aberdeen20111001_12
Aberdeen20111002_00
Aberdeen20111002_12
Aberdeen20111003_00
Aberdeen20111003_12
Aberdeen20111004_00


Aberdeen20120403_00
Aberdeen20120403_12
Aberdeen20120404_00
Aberdeen20120404_12
Aberdeen20120405_00
Aberdeen20120405_12
Aberdeen20120406_00
Aberdeen20120406_12
Aberdeen20120407_00
Aberdeen20120407_12
Aberdeen20120408_00
Aberdeen20120408_12
Aberdeen20120409_00
Aberdeen20120409_12
Aberdeen20120410_00
Aberdeen20120410_12
Aberdeen20120411_00
Aberdeen20120411_12
Aberdeen20120412_00
Aberdeen20120412_12
Aberdeen20120413_00
Aberdeen20120413_12
Aberdeen20120414_00
Aberdeen20120414_12
Aberdeen20120415_00
Aberdeen20120415_12
Aberdeen20120416_00
Aberdeen20120416_12
Aberdeen20120417_00
Aberdeen20120417_12
Aberdeen20120418_00
Aberdeen20120418_12
Aberdeen20120419_00
Aberdeen20120419_12
Aberdeen20120420_00
Aberdeen20120420_12
Aberdeen20120421_00
Aberdeen20120421_12
Aberdeen20120422_00
Aberdeen20120422_12
Aberdeen20120423_00
Aberdeen20120423_12
Aberdeen20120424_00
Aberdeen20120424_12
Aberdeen20120425_00
Aberdeen20120425_12
Aberdeen20120426_00
Aberdeen20120426_12
Aberdeen20120427_00
Aberdeen20120427_12


...

Aberdeen20170424_12
Aberdeen20170425_00
Aberdeen20170425_12
Aberdeen20170426_00
Aberdeen20170426_12
Aberdeen20170427_00
Aberdeen20170427_12
Aberdeen20170428_00
Aberdeen20170428_12
Aberdeen20170429_00
Aberdeen20170429_12
Aberdeen20170430_00
Aberdeen20170430_12
Aberdeen20170501_00
Aberdeen20170501_12
Aberdeen20170502_00
Aberdeen20170502_12
Aberdeen20170503_00
Aberdeen20170503_12
Aberdeen20170504_00
Aberdeen20170504_12
Aberdeen20170505_00
Aberdeen20170505_12
Aberdeen20170506_00
Aberdeen20170506_12
Aberdeen20170507_00
Aberdeen20170507_12
Aberdeen20170508_00
Aberdeen20170508_12
Aberdeen20170509_00
Aberdeen20170509_12
Aberdeen20170510_00
Aberdeen20170510_12
Aberdeen20170511_00
Aberdeen20170511_12
Aberdeen20170512_00
Aberdeen20170512_12
Aberdeen20170513_00
Aberdeen20170513_12
Aberdeen20170514_00
Aberdeen20170514_12
Aberdeen20170515_00
Aberdeen20170515_12
Aberdeen20170516_00
Aberdeen20170516_12
Aberdeen20170517_00
Aberdeen20170517_12
Aberdeen20170518_00
Aberdeen20170518_12
Aberdeen20170519_00


Aberdeen20171107_00
Aberdeen20171107_12
Aberdeen20171108_00
Aberdeen20171108_12
Aberdeen20171109_00
Aberdeen20171109_12
Aberdeen20171110_00
Aberdeen20171110_12
Aberdeen20171111_00
Aberdeen20171111_12
Aberdeen20171112_00
Aberdeen20171112_12
Aberdeen20171113_00
Aberdeen20171113_12
Aberdeen20171114_00
Aberdeen20171114_12
Aberdeen20171115_00
Aberdeen20171115_12
Aberdeen20171116_00
Aberdeen20171116_12
Aberdeen20171117_00
Aberdeen20171117_12
Aberdeen20171118_00
Aberdeen20171118_12
Aberdeen20171119_00
Aberdeen20171119_12
Aberdeen20171120_00
Aberdeen20171120_12
Aberdeen20171121_00
Aberdeen20171121_12
Aberdeen20171122_00
Aberdeen20171122_12
Aberdeen20171123_00
Aberdeen20171123_12
Aberdeen20171124_00
Aberdeen20171124_12
Aberdeen20171125_00
Aberdeen20171125_12
Aberdeen20171126_00
Aberdeen20171126_12
Aberdeen20171127_00
Aberdeen20171127_12
Aberdeen20171128_00
Aberdeen20171128_12
Aberdeen20171129_00
Aberdeen20171129_12
Aberdeen20171130_00
Aberdeen20171130_12
Aberdeen20171201_00
Aberdeen20171201_12


Aberdeen20180603_00
Aberdeen20180603_12
Aberdeen20180604_00
Aberdeen20180604_12
Aberdeen20180605_00
Aberdeen20180605_12
Aberdeen20180606_00
Aberdeen20180606_12
Aberdeen20180607_00
Aberdeen20180607_12
Aberdeen20180608_00
Aberdeen20180608_12
Aberdeen20180609_00
Aberdeen20180609_12
Aberdeen20180610_00
Aberdeen20180610_12
Aberdeen20180611_00
Aberdeen20180611_12
Aberdeen20180612_00
Aberdeen20180612_12
Aberdeen20180613_00
Aberdeen20180613_12
Aberdeen20180614_00
Aberdeen20180614_12
Aberdeen20180614_19
Aberdeen20180615_00
Aberdeen20180615_12
Aberdeen20180616_00
Aberdeen20180616_12
Aberdeen20180616_20
Aberdeen20180617_00
Aberdeen20180617_12
Aberdeen20180618_00
Aberdeen20180618_12
Aberdeen20180619_00
Aberdeen20180619_12
Aberdeen20180620_00
Aberdeen20180620_12
Aberdeen20180621_00
Aberdeen20180621_12
Aberdeen20180622_00
Aberdeen20180622_12
Aberdeen20180623_00
Aberdeen20180623_12
Aberdeen20180624_00
Aberdeen20180624_12
Aberdeen20180625_00
Aberdeen20180625_12
Aberdeen20180626_00
Aberdeen20180626_12


Aberdeen20181221_00
Aberdeen20181221_12
Aberdeen20181222_00
Aberdeen20181222_12
Aberdeen20181223_00
Aberdeen20181223_12
Aberdeen20181224_00
Aberdeen20181224_12
Aberdeen20181225_00
Aberdeen20181225_12
Aberdeen20181226_00
Aberdeen20181226_12
Aberdeen20181227_00
Aberdeen20181227_12
Aberdeen20181228_00
Aberdeen20181228_12
Aberdeen20181229_00
Aberdeen20181229_12
Aberdeen20181230_00
Aberdeen20181230_12
Aberdeen20181231_00
Aberdeen20181231_12
Aberdeen20190101_00
Aberdeen20190101_12
Aberdeen20190102_00
Aberdeen20190102_12
Aberdeen20190103_00
Aberdeen20190103_12
Aberdeen20190104_00
Aberdeen20190104_12
Aberdeen20190105_00
Aberdeen20190105_12
Aberdeen20190106_00
Aberdeen20190106_12
Aberdeen20190107_00
Aberdeen20190107_12
Aberdeen20190108_00
Aberdeen20190108_12
Aberdeen20190109_00
Aberdeen20190109_12
Aberdeen20190110_00
Aberdeen20190110_12
Aberdeen20190111_00
Aberdeen20190111_12
Aberdeen20190112_00
Aberdeen20190112_12
Aberdeen20190113_00
Aberdeen20190113_12
Aberdeen20190114_00
Aberdeen20190114_12


Aberdeen20190715_00
Aberdeen20190715_12
Aberdeen20190716_00
Aberdeen20190716_12
Aberdeen20190717_00
Aberdeen20190717_12
Aberdeen20190717_19
Aberdeen20190718_00
Aberdeen20190718_12
Aberdeen20190719_00
Aberdeen20190719_12
Aberdeen20190719_18
Aberdeen20190720_00
Aberdeen20190720_12
Aberdeen20190721_00
Aberdeen20190721_12
Aberdeen20190722_00
Aberdeen20190722_12
Aberdeen20190723_00
Aberdeen20190723_12
Aberdeen20190724_00
Aberdeen20190724_12
Aberdeen20190725_00
Aberdeen20190725_12
Aberdeen20190726_00
Aberdeen20190726_12
Aberdeen20190727_00
Aberdeen20190727_12
Aberdeen20190728_00
Aberdeen20190728_12
Aberdeen20190729_00
Aberdeen20190729_12
Aberdeen20190730_00
Aberdeen20190730_12
Aberdeen20190731_00
Aberdeen20190731_12
Aberdeen20190801_00
Aberdeen20190801_12
Aberdeen20190802_00
Aberdeen20190802_12
Aberdeen20190803_00
Aberdeen20190803_12
Aberdeen20190804_00
Aberdeen20190804_12
Aberdeen20190805_00
Aberdeen20190805_12
Aberdeen20190806_00
Aberdeen20190806_12
Aberdeen20190807_00
Aberdeen20190807_12
